# Постановка задачи

### Задача

- Парсинг данных по книгам с сайта 'https://www.litres.ru/popular/' и построение модели пресказания потенциального рейтинга книги по ее текстовому описанию

### Описание данных

**Источник данных:** https://www.litres.ru/popular/

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
import requests     
import numpy as np   
import pandas as pd  
import time          

In [4]:
from fake_useragent import UserAgent

In [5]:
from bs4 import BeautifulSoup

In [6]:
from fake_useragent import UserAgent

# 1. Парсинг

## 1.1 Проверка доступа к сайту

In [6]:
page_link = 'https://www.litres.ru/popular/'

response = requests.get(page_link, headers={'User-Agent': UserAgent().chrome})
response

<Response [200]>

In [7]:
response.request.headers

{'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.0.0 Safari/537.36', 'Accept-Encoding': 'gzip, deflate', 'Accept': '*/*', 'Connection': 'keep-alive'}

Посмотрим адекватно ли выгрузилось содержимое

In [8]:
html = response.content
soup = BeautifulSoup(html, 'html.parser')                        
print(soup.prettify()[:1000])

<!DOCTYPE html>
<html lang="ru">
 <script>
  if(typeof window.Intl==='undefined'||typeof window.Intl.DateTimeFormat==='undefined'||typeof window.MutationObserver==='undefined'||typeof window.IntersectionObserver==='undefined'||typeof window.ResizeObserver==='undefined'){document.write('<script src="https://polyfill.io/v3/polyfill.min.js?features=Intl.DateTimeFormat%2CMutationObserver%2CIntersectionObserver%2CResizeObserver"></scr'+'ipt>')}
 </script>
 <head>
  <meta charset="utf-8"/>
  <link href="https://www.litres.ru/static/litres/favicon.svg" rel="shortcut icon" type="image/svg+xml"/>
  <link href="https://www.litres.ru/static/litres/favicon.ico" rel="shortcut icon" type="image/x-icon"/>
  <link href="/rss/" rel="alternate" title="Новые книги на Литрес" type="application/rss+xml"/>
  <link href="https://www.litres.ru/static/litres/i/social/seo/icon_114x114.png?caf381" rel="apple-touch-icon"/>
  <link href="https://www.litres.ru/static/litres/inc/seo/yandex-tableau-manifest.json" rel

**Заключение**

- Доступ к странице есть;
- Данные выгрузились адекватно

## 2.1 Описание парсера

Сформируем скрипт выгрузки ссылок на книги со страницы

In [9]:
book_links = soup.findAll(lambda tag: tag.name == 'a' and tag.get('data-testid') == "art__title")
book_links = [link.attrs['href'] for link in book_links]

Проверим адекватна ли выгрузка

In [10]:
print(f'Всего ссылок выгружено: {len(book_links)}')
book_links[:10]

Всего ссылок выгружено: 36


['/book/tatyana-muzhickaya/teoriya-neveroyatnosti-kak-mechtat-chtoby-sbyvalos-kak-27304463/',
 '/book/luk-boden/hooponopono-drevniy-gavayskiy-metod-ispolneniya-zhelaniy-48428074/',
 '/book/ekaterina-kablukova/spasti-chudovische-70338325/?erid=LjN8K8pDB&source=art&adv_art_id=252&campaign_id=1709&promo_art_position=3&platform=web',
 '/audiobook/patrik-king/chitayte-ludey-kak-knigu-kak-analizirovat-ponimat-i-predskazyv-65834170/',
 '/book/sergey-lukyanenko/forsayt-70358980/',
 '/book/meri-li/tuman-polnoe-izdanie-70313815/',
 '/book/patrik-king/perestante-ugozhdat-ludyam-budte-assertivnym-perestante-zaboti-68002241/',
 '/book/erofey-trofimov/shatun-shag-v-neizvestnost-70336498/',
 '/book/anne-dar/rodnaya-krov-67066479/',
 '/book/mayk-omer/vnutri-ubiycy-47453330/']

Отлично, ссылки выгружаются. 
Оформим в виде функции

In [11]:
def getPageLinks(page_number):
    """
        Возвращает список ссылок на книги

        page_number: int/string
            номер страницы для парсинга

    """
    # составляем ссылку на страницу поиска
    page_link = 'https://www.litres.ru/popular/?page={}'.format(page_number)

    # запрашиваем данные по ней
    response = requests.get(page_link, headers={'User-Agent': UserAgent().chrome})

    if not response.ok:
        # если сервер нам отказал, вернем пустой лист для текущей страницы
        return []

    # получаем содержимое страницы и переводим в суп
    html = response.content
    soup = BeautifulSoup(html,'html.parser')

    # наконец, ищем ссылки на мемы и очищаем их от ненужных тэгов
    book_links = soup.findAll(lambda tag: tag.name == 'a' and tag.get('data-testid') == "art__title")
    book_links = ['https://www.litres.ru' + link.attrs['href'] for link in book_links]

    return book_links

Протестируем функцию и убедимся, что всё хорошо

In [13]:
book_links = getPageLinks(1)
book_links[:2]

['https://www.litres.ru/book/tatyana-muzhickaya/teoriya-neveroyatnosti-kak-mechtat-chtoby-sbyvalos-kak-27304463/',
 'https://www.litres.ru/book/luk-boden/hooponopono-drevniy-gavayskiy-metod-ispolneniya-zhelaniy-48428074/']

Отлично, функция работает

Теперь напишем функцию выгрузки информации о книге

In [14]:
def get_book_data(link):
    """
        Возвращает очищенное число просмотров/коментариев/...

        soup: объект bs4.BeautifulSoup
            суп текущей страницы

    """
    obj_dict = dict()
    
    # составляем ссылку на страницу поиска
    page_link = link

    # запрашиваем данные по ней
    response = requests.get(page_link, headers={'User-Agent': UserAgent().chrome})

    # получаем содержимое страницы и переводим в суп
    html = response.content
    soup = BeautifulSoup(html,'html.parser')
    
    try:
        obj = soup.find(lambda tag: tag.name == 'h1' and tag.get('itemprop') == "name")
        obj_dict["Наименование"] = obj.text
    except:
        obj_dict["Наименование"]=None
        
    try:
        obj = soup.find(lambda tag: tag.name == 'div' and tag.get('data-testid') == "book-short-description__wrapper")
        obj = obj.find(lambda tag: tag.name == 'div').text
        obj = str(obj).replace('\n', ' ')
        obj_dict["Краткое описание"] = obj
    except:
        obj_dict["Краткое описание"]=None   
     
    try:
        obj = soup.find(lambda tag: tag.name == 'meta' and tag.get('itemprop') == "ratingValue")
        obj_dict["Рейтинг"] = obj.attrs['content']
    except:
        obj_dict["Рейтинг"] = None
        
    try:
        obj = soup.find(lambda tag: tag.name == 'meta' and tag.get('itemprop') == "ratingCount")
        obj_dict["Количество оценок"] = obj.attrs['content']
    except:
        obj_dict["Количество оценок"] = None
        
    try:
        obj = soup.find(lambda tag: tag.name == 'div' and tag.get('data-testid') == "book-factoids__reviews")
        obj = obj.find(lambda tag: tag.name == 'span').text
        obj_dict["Количество отзывов"] = obj
    except:
        obj_dict["Количество отзывов"] = None
        
    try:
        obj = soup.find(lambda tag: tag.name == 'div' and tag.get('data-analytics-id') == "book-characteristics")
        obj = obj.findAll(lambda tag: tag.name == 'div' and tag.get('class') == ["CharacteristicsBlock-module__characteristic_2SKY6"])
        obj = [row for row in obj if 'Дата выхода на Литрес' in row.text][0]
        obj_dict["Дата публикации"] = obj.find(lambda tag: tag.name=='span' and 'Дата выхода на Литрес' not in tag.text).text
    except:
        obj_dict["Дата публикации"] = None
        
        
    obj_dict['page_link'] = link

    return obj_dict

Проверим работоспособность функции

In [15]:
book_data = get_book_data('https://www.litres.ru/audiobook/patrik-king/chitayte-ludey-kak-knigu-kak-analizirovat-ponimat-i-predskazyv-65834170/')
book_data

{'Наименование': 'Читайте людей как книгу. Как анализировать, понимать и предсказывать эмоции, мысли, намерения и поведение людей',
 'Краткое описание': 'В этой книге Патрика Кинга, автора мировых бестселлеров в области навыков социальной коммуникации, рассмотрены способы, с помощью которых можно развить умение «читать» и анализировать людей, их характер, поведение, мотивацию, невысказанные намерения. Этот навык бесценен в социуме. Задача автора – научить вас получать качественную, полезную, объективную информацию о людях во время первой же встречи, чтобы вы могли выстроить модель их мыслей, ощущений и действий в контексте ситуации. Читая эту книгу, вы узнаете, как определить, что вас обманывают или пытаются вами манипулировать, как выносить суждения о личностных характеристиках и ценностях людей, наблюдая за их речью, выражениями лиц, языком тела, и даже одеждой. Кроме того, знание, как «читать» людей, усилит вашу способность к состраданию, поможет стать лучшими родителями или партнер

Отлично, данные получены. Теперь оформим в виде Pandas DataFrame

In [16]:
final_df = pd.DataFrame(columns=['Наименование',  'Краткое описание', 'Рейтинг', 'Количество оценок', 'Дата публикации',
                                 'Количество отзывов', 'page_link'])

final_df = final_df.append(book_data, ignore_index=True).dropna(axis = 1)

In [17]:
final_df

,Наименование,Краткое описание,Рейтинг,Количество оценок,Дата публикации,Количество отзывов,page_link
0,"Читайте людей как книгу. Как анализировать, по...","В этой книге Патрика Кинга, автора мировых бес...",4.2,457,15 июля 2021,49,https://www.litres.ru/audiobook/patrik-king/ch...


Теперь выгрузим интересующую нас информацию о книгах:
   - Выгрузку будем производить с 1 по 40 страницы;
   - Каждую страницу будем сохранять в отдельном pandas.Dataframe на ПК в формате .pkl на случай обрыва связи.

In [ ]:
from tqdm import tqdm_notebook

for page_number in tqdm_notebook(range(1, 41), desc='Pages'):
    final_df = pd.DataFrame(columns=['Наименование',  'Краткое описание', 'Рейтинг', 'Количество оценок', 'Дата публикации',
                                 'Количество отзывов', 'page_link'])
    # Выгружаем все ссылки на книги со страницы
    book_links = getPageLinks(page_number)
    for book_links in tqdm_notebook(book_links, desc='Books', leave=False):
        # Делаем три попытки выгрузить данные.
        for i in range(3):
            try:
    
                data_row = get_book_data(book_links)
                final_df = final_df.append(data_row, ignore_index=True)
                
                # если всё получилось - выходим из внутреннего цикла
                break
            except:
                # Иначе, пробуем еще несколько раз, пока не закончатся попытки
                print('AHTUNG! parsing once again:', book_links)
                continue
            time.sleep(0.5)
    final_df.to_pickle(f'../data/parsing/df_page_{page_number}')

Проверим данные, которые мы спарсили

In [7]:
df=pd.DataFrame()
for page in range(41):
    df = pd.concat([df, pd.read_pickle(f'../data/parsing/df_page_{page}')], ignore_index=True)

In [8]:
df.head()

,Наименование,Краткое описание,Рейтинг,Количество оценок,Дата публикации,Количество отзывов,page_link
0,"Теория невероятности. Как мечтать, чтобы сбыва...","Никакой магии. Только здравый смысл, психологи...",4.8,1541,16 февраля 2019,269,https://www.litres.ru/book/tatyana-muzhickaya/...
1,"Читайте людей как книгу. Как анализировать, по...","В этой книге Патрика Кинга, автора мировых бес...",4.2,456,15 июля 2021,49,https://www.litres.ru/audiobook/patrik-king/ch...
2,Спасти чудовище,"Нищая швея и граф, чье лицо изуродовано войной...",4.9,466,14 февраля 2024,112,https://www.litres.ru/book/ekaterina-kablukova...
3,Форсайт,"Людям порой снится прошлое. Иногда хорошее, ин...",4.5,165,21 февраля 2024,23,https://www.litres.ru/book/sergey-lukyanenko/f...
4,Перестаньте угождать людям. Будьте ассертивным...,Угодничество не зря называют болезнью. Оно мож...,4.3,160,08 сентября 2022,32,https://www.litres.ru/book/patrik-king/peresta...
